# VacationPy

In [85]:
# import dependencies
import hvplot.pandas
import pandas as pd
import requests

# import api_key 
from api_keys import geoapify_key

In [86]:
# load the csv file created in part 1
city_data_df = pd.read_csv("../output_data/cities.csv")


city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,usogorsk,63.4106,48.6872,2.67,100,100,3.75,RU,1682351342
1,1,portland,45.5234,-122.6762,10.60,88,75,3.13,US,1682351215
2,2,novohrad-volynskyi,50.5941,27.6165,14.27,58,67,4.17,UA,1682351343
3,3,adamstown,-25.0660,-130.1015,24.73,88,98,5.71,PN,1682351343
4,4,ushuaia,-54.8000,-68.3000,6.81,76,75,1.54,AR,1682351225


## Step 1: Create a map that displays a point for every city in the `city_data_df` dataframe. The size of the point should be the humidity in each city.

In [87]:
%%capture --no-display

# Configure the map plot
humidity_map = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo = True,
    tiles = 'OSM',
    frame_width = 800,
    frame_height = 500,
    size = 'Humidity',
    color = 'City',
    alpha = 0.6
)

# Display the map
humidity_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

## Step 2: Narrow down the `city_data_df` dataframe to find your ideal weather location. 

Criteria
- max temp < 25 
- humidity < 65
- cloudiness < 50

In [88]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_df = city_data_df.loc[(city_data_df['Max Temp'] < 25) & (city_data_df['Humidity'] < 60) & (city_data_df['Cloudiness'] < 50)]
print(ideal_city_df.info())
# Drop any rows with null values
ideal_city_df = ideal_city_df.dropna()

# Display sample data
ideal_city_df

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64 entries, 25 to 558
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City_ID     64 non-null     int64  
 1   City        64 non-null     object 
 2   Lat         64 non-null     float64
 3   Lng         64 non-null     float64
 4   Max Temp    64 non-null     float64
 5   Humidity    64 non-null     int64  
 6   Cloudiness  64 non-null     int64  
 7   Wind Speed  64 non-null     float64
 8   Country     64 non-null     object 
 9   Date        64 non-null     int64  
dtypes: float64(4), int64(4), object(2)
memory usage: 5.5+ KB
None


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
25,25,badger,64.8000,-147.5333,-11.86,58,20,5.14,US,1682351349
54,54,old town,50.0870,14.4202,15.82,51,0,7.20,CZ,1682351358
59,59,al abyar,32.1900,20.5965,21.69,44,0,7.19,LY,1682351359
61,61,thompson,55.7435,-97.8558,-0.91,59,20,1.54,CA,1682351359
63,63,sandnessjoen,66.0217,12.6316,4.12,48,40,1.03,NO,1682351360
...,...,...,...,...,...,...,...,...,...,...
529,529,minas de marcona,-15.2119,-75.1103,22.07,59,2,6.52,PE,1682351454
530,530,laojunmiao,39.8333,97.7333,6.37,31,0,1.53,CN,1682351454
538,538,santa vitoria do palmar,-33.5189,-53.3681,24.66,59,44,3.72,BR,1682351455
539,539,zhangye,38.9342,100.4517,9.55,36,20,1.39,CN,1682351455


## Step 3: Create a new dataframe called `hotel_df`

In [89]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df.copy(deep=True)
hotel_df = hotel_df[["City", 'Country', 'Lat', 'Lng', 'Humidity']]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
25,badger,US,64.8000,-147.5333,58,
54,old town,CZ,50.0870,14.4202,51,
59,al abyar,LY,32.1900,20.5965,44,
61,thompson,CA,55.7435,-97.8558,59,
63,sandnessjoen,NO,66.0217,12.6316,48,
...,...,...,...,...,...,...
529,minas de marcona,PE,-15.2119,-75.1103,59,
530,laojunmiao,CN,39.8333,97.7333,31,
538,santa vitoria do palmar,BR,-33.5189,-53.3681,59,
539,zhangye,CN,38.9342,100.4517,36,


## Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 meters of your coordinates

In [90]:
# testing the json response so I understand the method of retreiving hotels from geoapify
base_url = "https://api.geoapify.com/v2/places"

params = {
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key,
    "limit": 20
}

zhangye_data = hotel_df.loc[538]
latitude = zhangye_data['Lat']
longitude = zhangye_data['Lng']
radius = 10000


params["filter"] = f"circle:{longitude},{latitude},{radius}"
params["bias"] = f"proximity:{longitude},{latitude}"



response = requests.get(base_url, params=params).json()
response

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'name': 'Hotel Brasil',
    'country': 'Brazil',
    'country_code': 'br',
    'region': 'South Region',
    'state': 'Rio Grande do Sul',
    'county': 'Região Geográfica Intermediária de Pelotas',
    'city': 'Santa Vitória do Palmar',
    'municipality': 'Região Geográfica Imediata de Pelotas',
    'postcode': '96230-000',
    'district': 'Santa Vitória do Palmar',
    'neighbourhood': 'Centro',
    'street': 'Rua Barão do Rio Branco',
    'housenumber': '703',
    'lon': -53.3658395,
    'lat': -33.5219953,
    'state_code': 'RS',
    'formatted': 'Hotel Brasil, Rua Barão do Rio Branco 703, Santa Vitória do Palmar - RS, 96230-000, Brazil',
    'address_line1': 'Hotel Brasil',
    'address_line2': 'Rua Barão do Rio Branco 703, Santa Vitória do Palmar - RS, 96230-000, Brazil',
    'categories': ['accommodation', 'accommodation.hotel'],
    'details': [],
    'datasource': {'sourcename': 'openstreetmap',

In [91]:
zhangye_data

City          santa vitoria do palmar
Country                            BR
Lat                          -33.5189
Lng                          -53.3681
Humidity                           59
Hotel Name                           
Name: 538, dtype: object

In [92]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = hotel_df.loc[index]['Lat']
    lng = hotel_df.loc[index]['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params).json()

    # Convert the API response to JSON format
    #name_address = # YOUR CODE HERE
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
badger - nearest hotel: Hotel North Pole
old town - nearest hotel: Terasa U Prince
al abyar - nearest hotel: No hotel found
thompson - nearest hotel: Thompson Inn
sandnessjoen - nearest hotel: Scandic Syv Søstre
nazareno - nearest hotel: No hotel found
shache - nearest hotel: 莎車新盛賓舘
gulf shores - nearest hotel: Beachside Resort
isafjordur - nearest hotel: Hótel Horn
bati - nearest hotel: No hotel found
kodiak - nearest hotel: Shelikof Lodge
mount isa - nearest hotel: Ibis Styles
flin flon - nearest hotel: Copperbelt Hotel
buka - nearest hotel: No hotel found
port augusta - nearest hotel: Hotel Commonwealth
korla - nearest hotel: Silver Star Hotel
happy valley-goose bay - nearest hotel: Hotel North Two
milas - nearest hotel: Eldehan Otel
cambria - nearest hotel: Olallieberry Inn
san julian - nearest hotel: No hotel found
broken hill - nearest hotel: Ibis Styles
puerto deseado - nearest hotel: Los Olmos
illapel - nearest hotel: Domingo Ortiz de Rosas
tura - nearest 

,City,Country,Lat,Lng,Humidity,Hotel Name
25,badger,US,64.8000,-147.5333,58,Hotel North Pole
54,old town,CZ,50.0870,14.4202,51,Terasa U Prince
59,al abyar,LY,32.1900,20.5965,44,No hotel found
61,thompson,CA,55.7435,-97.8558,59,Thompson Inn
63,sandnessjoen,NO,66.0217,12.6316,48,Scandic Syv Søstre
...,...,...,...,...,...,...
529,minas de marcona,PE,-15.2119,-75.1103,59,No hotel found
530,laojunmiao,CN,39.8333,97.7333,31,No hotel found
538,santa vitoria do palmar,BR,-33.5189,-53.3681,59,Hotel Brasil
539,zhangye,CN,38.9342,100.4517,36,Zhangye Hotel 张掖饭店


## Step 5: Add the hotel name and the country as additional information in the hover meassage for each city in the map

In [93]:
# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 500,
    size = 'Humidity',
    color = 'City',
    hover_cols = ['City', 'Hotel Name', 'Country']
)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)